<a href="https://colab.research.google.com/github/Neo-glitch/t.f-2.0-practice/blob/master/Reinforcement_Learning_for_Stock_Market_Trading(n_b_use_for_Live_Trading_at_your_own_risk).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall pandas-datareader -y
# Again, to be sure
!pip uninstall pandas-datareader -y
!pip install pandas-datareader -U

Found existing installation: pandas-datareader 0.10.0
Uninstalling pandas-datareader-0.10.0:
  Successfully uninstalled pandas-datareader-0.10.0
  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)


In [ ]:
#n.b use gpu when training, this shit takes long without it


import math  # to implement custom sigmoid function to scaler our stock prices
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader # to get and work with stock market info

from tqdm import tqdm_notebook, tqdm  # to viz progress in project
from collections import deque  # to implement experience replay, power of double ended que

In [ ]:
class AI_Trader():

  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):  # actions space are : Stay, Buy and Sell
    
    # init the class attr
    self.state_size = state_size       # states are previous days and stock prices over those days.
    self.action_space = action_space
    self.memory = deque(maxlen = 2000) # max element in deque
    self.inventory = []                # to hold all bought stocks
    self.model_name = model_name

    self.gamma = 0.95  # discount or gamma
    self.epsilon = 1.0  # beginning , action is random making epsilon 100%
    self.epsilon_final = 0.01 # final epsilon is 1%
    self.epsilon_decay = 0.995

    self.model = self.model_builder()


  def model_builder(self):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    # output layer to predictions, where we modify actions with a reward which is continous
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
  
    model.compile(loss = "mse", optimizer = tf.optimizers.Adam(lr = 0.001))

    return model


  def trade(self, state):
    """
    to return action to take
    """
    # if random gen number less than epsilon exec
    if(random.random() <= self.epsilon):
      return random.randrange(self.action_space)  # random num btw 0 & action_space.len
    
    # else choose best action based on prediction.
    actions = self.model.predict(state)
    return np.argmax(actions[0])


  def batch_train(self, batch_size):
    batch = []

    # since data is time constrained, to get latest data, we get batch from last n(batch_size) from mem
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done: # if agent not in terminated or dead state
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])  # normal reward formular

      target = self.model.predict(state)  # returns an action
      print(f"reward is: {reward}")
      print(f"target is: {target[0][action]}")

      target[0][action] = reward  # updates this action, with current reward

      self.model.fit(state, target, epochs = 1, verbose = 0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [ ]:
# helper functions


def sigmoid(x):
  """
  scales price 0 to 1 where 1 means stock price jump and vice versa
  """
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  """
  to print formatted price of stock bought or sold
  """
  print(f"price before formatting: {n}")
  if n < 0:
    return "- $ {0:2f}".format(abs(n))  # we lost money
  else:
    return "$ {0:2f}".format(abs(n))     # we gained money


def dataset_loader(stock_name= "AAPL"):
  """
  loads the needed dataset
  """
  # df holding stock data for stock_name passed
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_state = str(dataset.index[-1]).split()[0]

  close = dataset["Close"]

  return close


In [ ]:
dataset = data_reader.data.DataReader('TSLA', data_source="yahoo")

In [ ]:
dataset

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-04-25,62.796001,61.172001,61.599998,62.757999,33688500.0,62.757999
2017-04-26,62.900002,61.799999,62.473999,62.034000,23475000.0,62.034000
2017-04-27,62.618000,61.500000,62.338001,61.726002,17343000.0,61.726002
2017-04-28,62.959999,61.599998,61.966000,62.813999,22527500.0,62.813999
2017-05-01,65.449997,62.962002,62.976002,64.566002,44148000.0,64.566002
...,...,...,...,...,...,...
2022-04-18,1014.919983,973.409973,989.030029,1004.289978,17238400.0,1004.289978
2022-04-19,1034.939941,995.330017,1005.059998,1028.150024,16615900.0,1028.150024
2022-04-20,1034.000000,975.250000,1030.000000,977.200012,23570400.0,977.200012


In [ ]:
# helper to get state, from the data
def state_creator(data, timestep, window_size):
  """
  window_size = num previous days to consider before predicting a stock price
  data = stock market dataset
  timestep = day to predict the right move to take

  function converts days to diff btw days as state and target from stock price will be an action
  e.g BUY, SELL, STAY IN TRADE
  """
  starting_id = timestep - window_size + 1  # start state. +1 added since our state ain't price but diff btw prices of days

  if starting_id >= 0:
    # starting id is +ve
    windowed_data = data[starting_id:timestep+1]
  else:
    # starting id is negative, append first data as many times to match window size
    windowed_data = - starting_id * [data[0]] + list(data[0: timestep+1])

    state = []
    for i in range(window_size - 1):
      # to get state, which is price diff btw days normalized using helper sigmoid
      state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state])

In [ ]:
# Loading the dataset
stock_name = "AAPL"
data = dataset_loader(stock_name)
data

Date
2017-04-25     36.132500
2017-04-26     35.919998
2017-04-27     35.947498
2017-04-28     35.912498
2017-05-01     36.645000
                 ...    
2022-04-18    165.070007
2022-04-19    167.399994
2022-04-20    167.229996
2022-04-21    166.419998
2022-04-22    161.789993
Name: Close, Length: 1259, dtype: float64

In [ ]:
window_size = 10  # using 10 previous days to make predictions
episodes = 1000

batch_size = 32
data_samples = len(data) - 1   # -1 sine we want to predict last one

In [ ]:
# Define Trader model
trader = AI_Trader(window_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
trader.model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 32)                352       
                                                                 
 dense_21 (Dense)            (None, 64)                2112      
                                                                 
 dense_22 (Dense)            (None, 128)               8320      
                                                                 
 dense_23 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training Loop

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []  # to store all stocks bought
  
  # tqdm just shows progress bar in loops
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])  # add to my stock portfolio
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling(checks if stock already in portfolio)
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      # stop at end of data samples
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

Episode: 1/1000


  0%|          | 0/1258 [00:00<?, ?it/s]

price before formatting: 35.9474983215332
AI Trader bought:  $ 35.947498
price before formatting: 35.912498474121094
AI Trader bought:  $ 35.912498
price before formatting: 36.64500045776367
price before formatting: 0.6975021362304688
AI Trader sold:  $ 36.645000  Profit: $ 0.697502
price before formatting: 36.877498626708984
AI Trader bought:  $ 36.877499
price before formatting: 36.76499938964844
AI Trader bought:  $ 36.764999
price before formatting: 36.63249969482422
AI Trader bought:  $ 36.632500
price before formatting: 37.2400016784668
AI Trader bought:  $ 37.240002
price before formatting: 38.252498626708984
AI Trader bought:  $ 38.252499
price before formatting: 38.497501373291016
AI Trader bought:  $ 38.497501
price before formatting: 38.314998626708984
AI Trader bought:  $ 38.314999
price before formatting: 38.48749923706055
AI Trader bought:  $ 38.487499
price before formatting: 39.025001525878906
price before formatting: 3.1125030517578125
AI Trader sold:  $ 39.025002  Pro

  3%|▎         | 32/1258 [00:03<02:16,  9.01it/s]


ValueError: ignored